# Reference
https://www.geeksforgeeks.org/realtime-distance-estimation-using-opencv-python/

In [ ]:
import numpy as np
import cv2
import imutils

from matplotlib import pyplot as plt

In [ ]:
def find_marker(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    lower = 0.66 * np.mean(blur)
    upper = 1.33 * np.mean(blur)
    edged = cv2.Canny(blur, lower, upper)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    plt.imshow(edged, cmap=plt.cm.gray)
    
    c = max(cnts, key=cv2.contourArea)
    return cv2.minAreaRect(c)

In [ ]:
def distance_to_camera(known_width, focal_length, per_width):
    return (known_width * focal_length) / per_width

In [ ]:
KNOWN_DISTANCE = 24.0
KNOWN_WIDTH = 11.0


image = cv2.imread('../resources/distance_measure_sample1.jpg')
marker = find_marker(image)
focal_length = marker[1][0] * KNOWN_DISTANCE / KNOWN_WIDTH
distance = distance_to_camera(KNOWN_WIDTH, focal_length, marker[1][0])
distance

In [ ]:
def distance_finder(focal_length, real_face_width, face_width_in_frame):
    return (real_face_width * focal_length) * face_width_in_frame

def focal_length_finder(measured_distance, real_width, width_in_rf_image):
    focal_length = (width_in_rf_image * measured_distance) / real_width
    return focal_length

def face_data(image, face_detector):
    face_width = 0
    gray = cv2.cvtColor(image,  cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    
    for (x, y, h, w) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        face_width = w
        
    return face_width

In [ ]:
known_distance = 40.0
known_width = 14.0

face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

ref_image = cv2.imread('../resources/search-dataset2/matusoka yositsugu1.jpg')
ref_image_face_width = face_data(ref_image, face_detector)
focal_length_found = focal_length_finder(known_distance, known_width, ref_image_face_width)

print(focal_length_found)

In [ ]:
cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()
    face_width_in_frame = face_data(frame, face_detector)
    
    if face_width_in_frame != 0:
        dist = distance_finder(focal_length_found, known_width, face_width_in_frame)
        
        cv2.line(frame, (30, 30), (230, 30), (0, 0, 255), 32)
        cv2.line(frame, (30, 30), (230, 30), 0, 28)
        cv2.putText(frame, f"distance: {round(dist, 2)} CM", (30, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
